# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [19]:
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

from hyperopt import tpe,hp,Trials
from hyperopt.fmin import fmin

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [2]:
pth1 = '/datasets/autos.csv'
pth2 = '/Users/Саркисян Александр/Desktop/practicum_ds/projects/ЧислМетоды/autos.csv'
pth3 = '/Users/Alex/Desktop/practicum_ds/projects/ЧислМетоды/autos.csv'

if os.path.exists(pth1):
    autos = pd.read_csv(pth1)
elif os.path.exists(pth2):
    autos = pd.read_csv(pth2)
elif os.path.exists(pth3):
    autos = pd.read_csv(pth3)
else:
    print('Something is wrong')

**Информация о выборках:**

In [3]:
def about_df(df):
    display(df.head())
    display(df.info())
    display(df.isna().sum())
    display(df.describe())
    display(df.corr())
    print('Дубликаты =', df.duplicated().sum())

In [4]:
about_df(autos)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.026916,0.158872,-0.333199,0.110581,NaN,0.076055
RegistrationYear,0.026916,1.000000,-0.000828,-0.053447,-0.011619,NaN,-0.003459
Power,0.158872,-0.000828,1.000000,0.024002,0.043380,NaN,0.021665
Kilometer,-0.333199,-0.053447,0.024002,1.000000,0.009571,NaN,-0.007698
RegistrationMonth,0.110581,-0.011619,0.043380,0.009571,1.000000,NaN,0.013995
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.076055,-0.003459,0.021665,-0.007698,0.013995,NaN,1.000000


Дубликаты = 4


**Вывод:**

Итак, в таблице 16 столбецов.

Описание данных:

Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак
- Price — цена (евро)

Предварительно можно утверждать, что, данных достаточно для решения задачи.

При этом надо отметить, что в данных встречаются пропуски в категорийных признаках.

Соблюдение требований к стилю в названии колонок:

Отсутствие пробелов - да.

Использован «змеиный_регистр» (snake_case) или CamelCase - да.

Признаки:
- DateCrawled,
- RegistrationYear
- RegistrationMonth,
- DateCreated,
- NumberOfPictures,
- PostalCode,
- LastSeen,

не требуются для обучения модели т.к. не несут смысловой нагрузки.

Обнаружено 4 дубликата.

Обнаружены артефакты (аномалии) в следующих столбцах:
- Power

Чтобы двигаться дальше, нужно устранить проблемы в данных.

In [5]:
# Заполним пропуски в категорийных признаках модой, т.к. их доля в выборке составляют до 20%
columns_to_replace = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']

for column in columns_to_replace:
    autos[column] = autos[column].fillna(autos[column].mode()[0])

In [6]:
# из открытых источников известно, что мощность самого мощного автомобиля = 2000л.с.
autos.loc[autos['Power'] > 2000, 'Power'] = 2000

In [7]:
# нулевые значения заменим на медиану.
autos['Power'].median()

105.0

In [8]:
autos.loc[autos['Power'] == 0, 'Power'] = 105

In [9]:
# удаление дубликатов
autos = autos.drop_duplicates().reset_index(drop=True)

In [10]:
# удаление признаков
autos = autos.drop(['DateCrawled',
                    'DateCreated',
                    'RegistrationYear',
                    'RegistrationMonth',
                    'NumberOfPictures',
                    'PostalCode',
                    'LastSeen'], axis=1)

## Обучение моделей

**CatBoostRegressor**

In [13]:
# Формирование выборок
features_train, features_valid, target_train, target_valid = train_test_split(
    autos.drop('Price', axis=1), autos['Price'], test_size=0.25, random_state=12345) 


In [ ]:
seed=2
def objective(params):
    
    md=int(params['depth'])
    
        
    model=CatBoostRegressor(loss_function='RMSE',
                            iterations=500,
                            depth=md,
                            verbose=True,
                            cat_features=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'])
    model.fit(features_train, target_train)
    pred=model.predict(features_valid)
    score=mean_squared_error(target_valid, pred)**0.5
    return score

def optimize(trial):
    params={'depth':hp.uniform('depth', 5, 16)}
    
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=50,rstate=np.random.default_rng(seed))
    
    return best

trial=Trials()
best=optimize(trial)

In [15]:
best

{'depth': 15.789252093922688}

In [35]:
%%time
# CatBoostRegressor
cat_model = CatBoostRegressor(loss_function='RMSE', iterations=500, depth=16, verbose=100,
                              cat_features=['VehicleType', 'Gearbox', 'Model',
                                            'FuelType', 'Brand', 'NotRepaired'])
cat_model.fit(features_train, target_train)

Learning rate set to 0.173776
0:	learn: 4059.8123433	total: 1.25s	remaining: 10m 21s
100:	learn: 2015.0268931	total: 2m 23s	remaining: 9m 28s
200:	learn: 1859.2636770	total: 5m 1s	remaining: 7m 28s
300:	learn: 1773.1621770	total: 7m 40s	remaining: 5m 4s
400:	learn: 1721.5117953	total: 10m 25s	remaining: 2m 34s
499:	learn: 1676.9261459	total: 13m 13s	remaining: 0us
Wall time: 13min 18s


In [36]:
%%time
cat_prediction = cat_model.predict(features_valid)

Wall time: 579 ms


In [37]:
result = mean_squared_error(target_valid, cat_prediction)**0.5
print('RMSE на валидационной выборке для CatBoostRegressor:', result)

RMSE на валидационной выборке для CatBoostRegressor: 2170.641803873421


**RandomForestRegressor**

In [20]:
# Преобразуем категориальные признаки в численные техникой Ordinal Encoding (от англ. «кодирование по номеру категории»).
oe_autos = autos.copy()
encoder = LabelEncoder()
cat_columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

for column in cat_columns:
    oe_autos[column] = encoder.fit_transform(oe_autos[column])

In [21]:
# Формирование выборок для OE
features_train_oe, features_valid_oe, target_train_oe, target_valid_oe = train_test_split(
    oe_autos.drop('Price', axis=1), oe_autos['Price'], test_size=0.25, random_state=12345) 

In [40]:
seed=2
def objective(params):
    
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    msl=int(params['min_samples_leaf'])
    mss=int(params['min_samples_split'])
    
    model=RandomForestRegressor(n_estimators=est,max_depth=md,min_samples_leaf=msl,min_samples_split=mss)
    model.fit(features_train_oe, target_train_oe)
    pred=model.predict(features_valid_oe)
    score=mean_squared_error(target_valid_oe, pred)**0.5
    return score

def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators',10, 50),
           'max_depth':hp.uniform('max_depth',5,20),
           'min_samples_leaf':hp.uniform('min_samples_leaf',1,5),
           'min_samples_split':hp.uniform('min_samples_split',2,6)}
    
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=50,rstate=np.random.default_rng(seed))
    
    return best

trial=Trials()
best=optimize(trial)

100%|█████████████████████████████████████████████████| 50/50 [08:12<00:00,  9.86s/trial, best loss: 2225.344971776495]


In [41]:
best

{'max_depth': 19.46949038542492,
 'min_samples_leaf': 2.9015650037511853,
 'min_samples_split': 4.091488507593088,
 'n_estimators': 49.46715676536605}

In [42]:
%%time
# RandomForestRegressor with best params
rf_model = RandomForestRegressor(criterion = 'mse',
                                 random_state=12345,
                                 n_estimators = 49,
                                 max_depth = 19,
                                 min_samples_leaf = 3,
                                 min_samples_split = 4)
rf_model.fit(features_train_oe, target_train_oe)

Wall time: 15.5 s


RandomForestRegressor(max_depth=19, min_samples_leaf=3, min_samples_split=4,
                      n_estimators=49, random_state=12345)

In [43]:
%%time
rf_prediction = rf_model.predict(features_valid_oe)

Wall time: 731 ms


In [44]:
result = mean_squared_error(target_valid_oe, rf_prediction)**0.5
print('RMSE на валидационной выборке для RandomForestRegressor:', result)

RMSE на валидационной выборке для RandomForestRegressor: 2231.0010473252414


**LGBMRegressor**

In [22]:
# Преобразуем категориальные признаки в тип category.
LGBM_autos = autos.copy()
LGBM_autos = LGBM_autos.astype({'VehicleType':'category', 'Gearbox':'category', 'Model':'category',
                                'FuelType':'category', 'Brand':'category', 'NotRepaired':'category'})

In [23]:
# Формирование выборок для LGBM
features_train_LGBM, features_valid_LGBM, target_train_LGBM, target_valid_LGBM = train_test_split(
    LGBM_autos.drop('Price', axis=1), LGBM_autos['Price'], test_size=0.25, random_state=12345) 

In [26]:
seed=2
def objective(params):
    
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    
    
    model=LGBMRegressor(n_estimators=est,max_depth=md)
    model.fit(features_train_LGBM, target_train_LGBM)
    pred=model.predict(features_valid_LGBM)
    score=mean_squared_error(target_valid_LGBM, pred)**0.5
    return score

def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators', 400, 1000),
           'max_depth':hp.uniform('max_depth',5,20)}
    
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=50,rstate=np.random.default_rng(seed))
    
    return best

trial=Trials()
best=optimize(trial)

100%|█████████████████████████████████████████████████| 50/50 [04:27<00:00,  5.34s/trial, best loss: 2162.686008938701]


In [27]:
best

{'max_depth': 13.332910720181824, 'n_estimators': 995.8255201762603}

In [28]:
%%time
model = LGBMRegressor(n_estimators=996,max_depth=13)
model.fit(features_train_LGBM, target_train_LGBM)

Wall time: 4.02 s


LGBMRegressor(max_depth=13, n_estimators=996)

In [29]:
%%time
lgbm_prediction = model.predict(features_valid_LGBM)

Wall time: 1.01 s


In [30]:
result = mean_squared_error(target_valid_LGBM, lgbm_prediction)**0.5
print('RMSE на валидационной выборке для LGBMRegressor:', result)

RMSE на валидационной выборке для LGBMRegressor: 2162.659501449274


**LinearRegression**

In [24]:
# Преобразуем категориальные признаки в численные техникой прямого кодирования,
# или отображения (англ. One-Hot Encoding, OHE).
ohe_autos = autos.copy()
ohe_autos = pd.get_dummies(ohe_autos, drop_first=True)

In [25]:
# Формирование выборок для OHE
features_train_ohe, features_valid_ohe, target_train_ohe, target_valid_ohe = train_test_split(
    ohe_autos.drop('Price', axis=1), ohe_autos['Price'], test_size=0.25, random_state=12345) 

In [31]:
%%time
# LinearRegression
lr_model = LinearRegression(normalize=True)
lr_model.fit(features_train_ohe, target_train_ohe)

Wall time: 4.73 s


LinearRegression(normalize=True)

In [32]:
%%time
lr_prediction = lr_model.predict(features_valid_ohe)

Wall time: 179 ms


In [33]:
result = mean_squared_error(target_valid_ohe, lr_prediction)**0.5
print('RMSE на валидационной выборке для LinearRegression:', result)

RMSE на валидационной выборке для LinearRegression: 3163.5077903810384


## Анализ моделей

**С точки зрения параметров скорость-качество то лучшие показатели у LGBMRegressor.**

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей